In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import random

pd.set_option('display.max_columns',None)

In [3]:
# Import dataset
data = pd.read_csv('/content/drive/MyDrive/DSCI 560 Project/Data/combined_rest_health_0331.csv')
data.head()

,FACILITY_NAME,FACILITY_ADDRESS,FACILITY_CITY,FACILITY_ID,FACILITY_STATE,FACILITY_ZIP,OWNER_ID,PE_DESCRIPTION,SCORE,VIOLATION_CODE,VIOLATION_DESCRIPTION,type,size,risk_level,ID,restaurant_id,url,name,address,score,review_counts,price,category,open_time,amenities,open_hours_week,new_address,state,zipcode,city,num_photos,is_bus_web,is_phone_number,is_message_bus,num_attributes,num_questions,comments_list
0,#1 CAFE,2080 CENTURY PARK E STE 108,LOS ANGELES,FA0056432,CA,90067,OW0000002,RESTAURANT (0-30) SEATS HIGH RISK,90.521739,"F014,F007,F033,F036,F027,F035,F034,F049,F040,F...",# 14. Food contact surfaces: clean and sanitiz...,restaurant,0-30,HIGH,0,753,https://www.yelp.com/biz/one-cafe-los-angeles-...,One Cafe,"2080 Century Park E Los Angeles, CA 90067",3.5,7 reviews,$$,Cafes,NaN,Amenities and More\r\nHealth ScoreA\r\nOffers ...,NaN,2080 Century Park E Los Angeles,CA,90067,Los Angeles,See all 18 photos,False,True,False,4.0,NaN,"[""Hi, it's Pauly P here dropping a bomb review..."
1,#2 MOON BBQ,478 N WESTERN AVE,LOS ANGELES,FA0002817,CA,90004,OW0024959,RESTAURANT (61-150) SEATS HIGH RISK,91.880952,"F013,F037,F033,F044,F033,F014,F046,F035,F033,F...","# 13. Food in good condition, safe and unadult...",restaurant,61-150,HIGH,4,2526,https://www.yelp.com/biz/moon-bbq-2-los-angele...,Moon BBQ 2,"478 N Western Ave Los Angeles, CA 90004",3.0,838 reviews,$$,"Korean, Barbeque",Mon\n11:00 AM - 2:00 AM (Next day)\nTue\n11:00...,Amenities and More\nHealth ScoreA\nNo Reservat...,NaN,478 N Western Ave Los Angeles,CA,90004,Los Angeles,See all 566 photos,True,True,False,19.0,See all 16 questions,"[""This is my go-to spot for cheap kbbq eats. T..."
2,101 ASIAN KITCHEN INC,7170 BEVERLY BLVD,LOS ANGELES,FA0224563,CA,90036,OW0229023,RESTAURANT (0-30) SEATS HIGH RISK,90.894737,"F009,F052,F036,F030,F053,F037,F039,F033,F044,F...","# 09. Proper cooling methods,# 01b. Food safet...",restaurant,0-30,HIGH,15,1080,https://www.yelp.com/biz/101-asian-kitchen-los...,101 Asian Kitchen,"7170 Beverly Blvd Los Angeles, CA 90036",3.5,232 reviews,$$,"Chinese, Sushi Bars, Hawaiian",Mon\r\n3:00 PM - 1:00 AM (Next day)\r\nTue\r\n...,Amenities and More\r\nHealth ScoreA\r\nOffers ...,NaN,7170 Beverly Blvd Los Angeles,CA,90036,Los Angeles,See all 118 photos,True,True,False,16.0,NaN,"[""Hey why not get yourself some Chinese Food (..."
3,WOK MASTER,7170 BEVERLY BLVD,LOS ANGELES,FA0175504,CA,90036,OW0182054,RESTAURANT (0-30) SEATS HIGH RISK,90.000000,"F033,F036,F037,F040,F039,F023,F004,F044",# 33. Nonfood-contact surfaces clean and in go...,restaurant,0-30,HIGH,13009,1080,https://www.yelp.com/biz/101-asian-kitchen-los...,101 Asian Kitchen,"7170 Beverly Blvd Los Angeles, CA 90036",3.5,232 reviews,$$,"Chinese, Sushi Bars, Hawaiian",Mon\r\n3:00 PM - 1:00 AM (Next day)\r\nTue\r\n...,Amenities and More\r\nHealth ScoreA\r\nOffers ...,NaN,7170 Beverly Blvd Los Angeles,CA,90036,Los Angeles,See all 118 photos,True,True,False,16.0,NaN,"[""Hey why not get yourself some Chinese Food (..."
4,24 HOUR FITNESS #122,5959 W CENTURY BLVD,LOS ANGELES,FA0139042,CA,90045,OW0000732,"FOOD MKT RETAIL (2,000+ SF) LOW RISK",99.000000,F033,# 33. Nonfood-contact surfaces clean and in go...,food market,"2,000+ SF",LOW,34,7523,https://www.yelp.com/biz/5959-kitchen-and-bar-...,5959 Kitchen and Bar,"5959 W Century Blvd Los Angeles, CA 90045",5.0,2 reviews,NaN,"Cocktail Bars, American (Traditional)",Mon\n11:00 AM - 11:00 PM\nTue\n11:00 AM - 11:0...,Amenities and More\nHealth ScoreA\nOffers Take...,84.0,5959 W Century Blvd Los Angeles,CA,90045,Los Angeles,See all 5 photos,False,True,True,20.0,NaN,['The food is fantastic and the price is right...


In [4]:
# get unique categories
def get_category(data):
    categories = set()
    for x in list(data.category):
        # 'Korean, Barbeque'
        clists = x.split(',')
        # print(clists)
        for c in clists:
            c = c.replace(' ','')
            categories.add(c)
    return list(categories)

categories = get_category(data)

In [5]:
# categories

In [6]:
# select restaurants from df as df1, use explicit filter
  # price, risk, amenities(delivery, takeout, reservation, vegetarian, group, parking, wheelchair accessible)
# select restaurants from df1 as df2, use fuzzy filter
  # location accroding to zipcode, name
# if cnt >10, over
# if cnt <10, calculate distance between df1 and df2 until cnt=10

In [7]:
# split amenities
def check_delivery(row):
  if type(row["amenities"])!=type(np.nan):
      if 'Offers Delivery' in row['amenities']:
        return 1
      elif 'No Delivery' in row['amenities']:
        return 0
      else:
        return np.nan

def check_takeout(row):
  if type(row["amenities"])!=type(np.nan):
      if 'Offers Takeout' in row['amenities']:
        return 1
      elif 'No Takeout' in row['amenities']:
        return 0
      else:
        return np.nan

def check_reservation(row):
    if type(row["amenities"])!=type(np.nan):
      if 'Takes Reservations' in row['amenities']:
        return 1
      elif 'No Reservations' in row['amenities']:
        return 0
      else:
        return np.nan

def check_vegetarian(row):
    if type(row["amenities"])!=type(np.nan):
      if 'Limited Vegetarian Options' in row['amenities'] or 'Many Vegetarian Options' in row['amenities']:
        return 1
      elif 'No Vegetarian Options' in row['amenities']:
        return 0
      else:
        return np.nan

def check_groups(row):
    if type(row["amenities"])!=type(np.nan):
      if 'Not Good For Groups' in row['amenities']:
        return 0
      elif 'Good for Groups' in row['amenities']:
        return 1
      else:
        return np.nan

def check_wheelchair(row):
    if type(row["amenities"])!=type(np.nan):
      if 'Wheelchair Accessible' in row['amenities']:
        return 1
      else:
        return 0

data['delivery'] = data.apply(lambda row: check_delivery(row), axis = 1)
data['takeout'] = data.apply(lambda row: check_takeout(row), axis = 1)
data['groups'] = data.apply(lambda row: check_groups(row), axis = 1)
data['vegetarian'] = data.apply(lambda row: check_vegetarian(row), axis = 1)
data['reservation'] = data.apply(lambda row: check_reservation(row), axis = 1)
data['wheelchair'] = data.apply(lambda row: check_wheelchair(row), axis = 1)
data.head()

,FACILITY_NAME,FACILITY_ADDRESS,FACILITY_CITY,FACILITY_ID,FACILITY_STATE,FACILITY_ZIP,OWNER_ID,PE_DESCRIPTION,SCORE,VIOLATION_CODE,VIOLATION_DESCRIPTION,type,size,risk_level,ID,restaurant_id,url,name,address,score,review_counts,price,category,open_time,amenities,open_hours_week,new_address,state,zipcode,city,num_photos,is_bus_web,is_phone_number,is_message_bus,num_attributes,num_questions,comments_list,delivery,takeout,groups,vegetarian,reservation,wheelchair
0,#1 CAFE,2080 CENTURY PARK E STE 108,LOS ANGELES,FA0056432,CA,90067,OW0000002,RESTAURANT (0-30) SEATS HIGH RISK,90.521739,"F014,F007,F033,F036,F027,F035,F034,F049,F040,F...",# 14. Food contact surfaces: clean and sanitiz...,restaurant,0-30,HIGH,0,753,https://www.yelp.com/biz/one-cafe-los-angeles-...,One Cafe,"2080 Century Park E Los Angeles, CA 90067",3.5,7 reviews,$$,Cafes,NaN,Amenities and More\r\nHealth ScoreA\r\nOffers ...,NaN,2080 Century Park E Los Angeles,CA,90067,Los Angeles,See all 18 photos,False,True,False,4.0,NaN,"[""Hi, it's Pauly P here dropping a bomb review...",1.0,1.0,0.0,NaN,0.0,0.0
1,#2 MOON BBQ,478 N WESTERN AVE,LOS ANGELES,FA0002817,CA,90004,OW0024959,RESTAURANT (61-150) SEATS HIGH RISK,91.880952,"F013,F037,F033,F044,F033,F014,F046,F035,F033,F...","# 13. Food in good condition, safe and unadult...",restaurant,61-150,HIGH,4,2526,https://www.yelp.com/biz/moon-bbq-2-los-angele...,Moon BBQ 2,"478 N Western Ave Los Angeles, CA 90004",3.0,838 reviews,$$,"Korean, Barbeque",Mon\n11:00 AM - 2:00 AM (Next day)\nTue\n11:00...,Amenities and More\nHealth ScoreA\nNo Reservat...,NaN,478 N Western Ave Los Angeles,CA,90004,Los Angeles,See all 566 photos,True,True,False,19.0,See all 16 questions,"[""This is my go-to spot for cheap kbbq eats. T...",0.0,0.0,1.0,1.0,0.0,0.0
2,101 ASIAN KITCHEN INC,7170 BEVERLY BLVD,LOS ANGELES,FA0224563,CA,90036,OW0229023,RESTAURANT (0-30) SEATS HIGH RISK,90.894737,"F009,F052,F036,F030,F053,F037,F039,F033,F044,F...","# 09. Proper cooling methods,# 01b. Food safet...",restaurant,0-30,HIGH,15,1080,https://www.yelp.com/biz/101-asian-kitchen-los...,101 Asian Kitchen,"7170 Beverly Blvd Los Angeles, CA 90036",3.5,232 reviews,$$,"Chinese, Sushi Bars, Hawaiian",Mon\r\n3:00 PM - 1:00 AM (Next day)\r\nTue\r\n...,Amenities and More\r\nHealth ScoreA\r\nOffers ...,NaN,7170 Beverly Blvd Los Angeles,CA,90036,Los Angeles,See all 118 photos,True,True,False,16.0,NaN,"[""Hey why not get yourself some Chinese Food (...",1.0,1.0,1.0,NaN,0.0,1.0
3,WOK MASTER,7170 BEVERLY BLVD,LOS ANGELES,FA0175504,CA,90036,OW0182054,RESTAURANT (0-30) SEATS HIGH RISK,90.000000,"F033,F036,F037,F040,F039,F023,F004,F044",# 33. Nonfood-contact surfaces clean and in go...,restaurant,0-30,HIGH,13009,1080,https://www.yelp.com/biz/101-asian-kitchen-los...,101 Asian Kitchen,"7170 Beverly Blvd Los Angeles, CA 90036",3.5,232 reviews,$$,"Chinese, Sushi Bars, Hawaiian",Mon\r\n3:00 PM - 1:00 AM (Next day)\r\nTue\r\n...,Amenities and More\r\nHealth ScoreA\r\nOffers ...,NaN,7170 Beverly Blvd Los Angeles,CA,90036,Los Angeles,See all 118 photos,True,True,False,16.0,NaN,"[""Hey why not get yourself some Chinese Food (...",1.0,1.0,1.0,NaN,0.0,1.0
4,24 HOUR FITNESS #122,5959 W CENTURY BLVD,LOS ANGELES,FA0139042,CA,90045,OW0000732,"FOOD MKT RETAIL (2,000+ SF) LOW RISK",99.000000,F033,# 33. Nonfood-contact surfaces clean and in go...,food market,"2,000+ SF",LOW,34,7523,https://www.yelp.com/biz/5959-kitchen-and-bar-...,5959 Kitchen and Bar,"5959 W Century Blvd Los Angeles, CA 90045",5.0,2 reviews,NaN,"Cocktail Bars, American (Traditional)",Mon\n11:00 AM - 11:00 PM\nTue\n11:00 AM - 11:0...,Amenities and More\nHealth ScoreA\nOffers Take...,84.0,5959 W Century Blvd Los Angeles,CA,90045,Los Angeles,See all 5 photos,False,True,True,20.0,NaN,['The food is fantastic and the price is right...,0.0,1.0,NaN,NaN,NaN,1.0


In [8]:
# Extract key features
data_key = data[['name', 'SCORE', 'size', 'risk_level', 'url', 'score', 'review_counts', 'price', 'category',
                 'delivery', 'takeout', 'groups', 'vegetarian', 'reservation','wheelchair',"address","zipcode"]]
data_key.head()

,name,SCORE,size,risk_level,url,score,review_counts,price,category,delivery,takeout,groups,vegetarian,reservation,wheelchair,address,zipcode
0,One Cafe,90.521739,0-30,HIGH,https://www.yelp.com/biz/one-cafe-los-angeles-...,3.5,7 reviews,$$,Cafes,1.0,1.0,0.0,NaN,0.0,0.0,"2080 Century Park E Los Angeles, CA 90067",90067
1,Moon BBQ 2,91.880952,61-150,HIGH,https://www.yelp.com/biz/moon-bbq-2-los-angele...,3.0,838 reviews,$$,"Korean, Barbeque",0.0,0.0,1.0,1.0,0.0,0.0,"478 N Western Ave Los Angeles, CA 90004",90004
2,101 Asian Kitchen,90.894737,0-30,HIGH,https://www.yelp.com/biz/101-asian-kitchen-los...,3.5,232 reviews,$$,"Chinese, Sushi Bars, Hawaiian",1.0,1.0,1.0,NaN,0.0,1.0,"7170 Beverly Blvd Los Angeles, CA 90036",90036
3,101 Asian Kitchen,90.000000,0-30,HIGH,https://www.yelp.com/biz/101-asian-kitchen-los...,3.5,232 reviews,$$,"Chinese, Sushi Bars, Hawaiian",1.0,1.0,1.0,NaN,0.0,1.0,"7170 Beverly Blvd Los Angeles, CA 90036",90036
4,5959 Kitchen and Bar,99.000000,"2,000+ SF",LOW,https://www.yelp.com/biz/5959-kitchen-and-bar-...,5.0,2 reviews,NaN,"Cocktail Bars, American (Traditional)",0.0,1.0,NaN,NaN,NaN,1.0,"5959 W Century Blvd Los Angeles, CA 90045",90045


In [9]:
# Add a column for tags
def combine_tags(row):
  tags = []
  tags.append(str(row['size']))
  tags.append(str(row['risk_level']))
  tags.append(str(row['price']))
  for category in row['category'].split(', '):
    tags.append(str(category))
  tags.append('offers delivery' if row['delivery'] ==  1 else 'no delivery')
  tags.append('offers takeout' if row['takeout'] ==  1 else 'no takeout')
  tags.append('good for groups' if row['groups'] ==  1 else 'not good for groups')
  tags.append('offers vegetarian options' if row['vegetarian'] ==  1 else 'no vegetarian options')
  tags.append('offers reservation' if row['reservation'] ==  1 else 'no reservation')
  tags.append('wheelchair accessible' if row['reservation'] ==  1 else 'wheelchair not accessible')
  tags.append(str(row['zipcode']))
  return tags

data_key['tags'] = data_key.apply(lambda row: combine_tags(row), axis = 1)
data_key.head()

<ipython-input-9-0d8afca2c999>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_key['tags'] = data_key.apply(lambda row: combine_tags(row), axis = 1)


,name,SCORE,size,risk_level,url,score,review_counts,price,category,delivery,takeout,groups,vegetarian,reservation,wheelchair,address,zipcode,tags
0,One Cafe,90.521739,0-30,HIGH,https://www.yelp.com/biz/one-cafe-los-angeles-...,3.5,7 reviews,$$,Cafes,1.0,1.0,0.0,NaN,0.0,0.0,"2080 Century Park E Los Angeles, CA 90067",90067,"[0-30, HIGH, $$, Cafes, offers delivery, offer..."
1,Moon BBQ 2,91.880952,61-150,HIGH,https://www.yelp.com/biz/moon-bbq-2-los-angele...,3.0,838 reviews,$$,"Korean, Barbeque",0.0,0.0,1.0,1.0,0.0,0.0,"478 N Western Ave Los Angeles, CA 90004",90004,"[61-150, HIGH, $$, Korean, Barbeque, no delive..."
2,101 Asian Kitchen,90.894737,0-30,HIGH,https://www.yelp.com/biz/101-asian-kitchen-los...,3.5,232 reviews,$$,"Chinese, Sushi Bars, Hawaiian",1.0,1.0,1.0,NaN,0.0,1.0,"7170 Beverly Blvd Los Angeles, CA 90036",90036,"[0-30, HIGH, $$, Chinese, Sushi Bars, Hawaiian..."
3,101 Asian Kitchen,90.000000,0-30,HIGH,https://www.yelp.com/biz/101-asian-kitchen-los...,3.5,232 reviews,$$,"Chinese, Sushi Bars, Hawaiian",1.0,1.0,1.0,NaN,0.0,1.0,"7170 Beverly Blvd Los Angeles, CA 90036",90036,"[0-30, HIGH, $$, Chinese, Sushi Bars, Hawaiian..."
4,5959 Kitchen and Bar,99.000000,"2,000+ SF",LOW,https://www.yelp.com/biz/5959-kitchen-and-bar-...,5.0,2 reviews,NaN,"Cocktail Bars, American (Traditional)",0.0,1.0,NaN,NaN,NaN,1.0,"5959 W Century Blvd Los Angeles, CA 90045",90045,"[2,000+ SF, LOW, nan, Cocktail Bars, American ..."


In [10]:
data_key.sample(5)

,name,SCORE,size,risk_level,url,score,review_counts,price,category,delivery,takeout,groups,vegetarian,reservation,wheelchair,address,zipcode,tags
2139,AOC Brentwood,92.350000,151 +,HIGH,https://www.yelp.com/biz/aoc-brentwood-los-ang...,3.5,1289 reviews,$$$,"American (New), Breakfast & Brunch",1.0,1.0,1.0,1.0,1.0,1.0,"11648 San Vicente Blvd Los Angeles, CA 90049",90049,"[151 + , HIGH, $$$, American (New), Breakfast ..."
394,Dumpling House,87.568182,0-30,HIGH,https://www.yelp.com/biz/dumpling-house-los-an...,4.0,647 reviews,$$,"Korean, Noodles, Specialty Food",NaN,NaN,1.0,NaN,0.0,0.0,"3525 W 3rd St Los Angeles, CA 90020",90020,"[0-30, HIGH, $$, Korean, Noodles, Specialty Fo..."
241,Awoolim,93.261905,0-30,HIGH,https://www.yelp.com/biz/awoolim-los-angeles?o...,4.0,209 reviews,$,Korean,1.0,1.0,1.0,1.0,0.0,0.0,928 S Western Ave Ste 125 Koreatown Plaza Los ...,90006,"[0-30, HIGH, $, Korean, offers delivery, offer..."
1186,Here's Looking At You,96.473684,0-30,HIGH,https://www.yelp.com/biz/heres-looking-at-you-...,4.0,558 reviews,$$$,American (Traditional),0.0,0.0,0.0,1.0,1.0,1.0,"3901 W 6th St Los Angeles, CA 90020",90020,"[0-30, HIGH, $$$, American (Traditional), no d..."
1833,Pho 24,88.687500,0-30,HIGH,https://www.yelp.com/biz/pho-24-los-angeles?os...,4.0,815 reviews,$$,"Vietnamese, Korean, Soup",1.0,1.0,1.0,1.0,0.0,0.0,"3450 W 6th St Ste 109-B Los Angeles, CA 90020",90020,"[0-30, HIGH, $$, Vietnamese, Korean, Soup, off..."


In [22]:
# Demo #1 - recommendation based on features

# Find restaurants that contain all input features
def exact_match(df, tags):
  candidate_restaurant = []
  candidate_pairs = []
  for i in range(len(df)):
    restaurant = df.iloc[i]
    if tags.issubset(set(restaurant['tags'])) and restaurant['name'] not in candidate_restaurant:
      candidate_restaurant.append(restaurant['name'])
      candidate_pairs.append((restaurant['name'], restaurant['url'], float(1.0)))
  return candidate_restaurant, candidate_pairs


# Remove restaurants that are already in candidate_pairs
def remaining_df(df, candidate_restaurant):
  final_df = df[df.name.isin(candidate_restaurant) == False]
  return final_df


# Calculate jaccard similarity
def jaccard_similarity(row, tags):
  intersection = set(row['tags']).intersection(tags)
  union = set(row['tags']).union(tags)
  res = round(len(intersection)/len(union), 4)
  return res


# If len(candidate_pairs) < 10, choose remaining candidates by calculating jaccard similarities
def jaccard_match(df, tags, candidate_pairs):

  # Calculate jaccard similarity between input tags and each restaurant tags
  df['jaccard'] = df.apply(lambda row: jaccard_similarity(row, tags), axis = 1)
  final_df = df.sort_values(by = ['jaccard'], ascending = False).reset_index(drop = True)

  # Fill the remaining spots using the top 10-len(candidates) jaccard similarities
  i = 0
  curr_jaccard_candidates = []
  curr_jaccard = final_df.iloc[0]['jaccard']

  while i < len(final_df) and len(candidate_pairs) < 10:
    # If there are ties, record a list of restaurants with the current jaccard similarity
    if final_df.iloc[i]['jaccard'] == curr_jaccard:
      curr_jaccard_candidates.append((final_df.iloc[i]['name'], final_df.iloc[i]['url'], final_df.iloc[i]['jaccard']))
      i += 1
    # As soon as a different, lower jaccard similarity appears
    else:
      # If the recorded list can fit into the remaining spots of candidate_pairs, add all pairs in
      if len(curr_jaccard_candidates) <= 10 - len(candidate_pairs):
        for pair in curr_jaccard_candidates:
          if pair not in candidate_pairs:
            candidate_pairs.append(pair)
      # Else choose a random sample that can fit into the remaining spots
      else:
        new_candidates = random.sample(curr_jaccard_candidates, 10 - len(candidate_pairs))
        for pair in new_candidates:
          if pair not in candidate_pairs:
            candidate_pairs.append(pair)

      # Reset that different, lower jaccard similariy as the current value
      curr_jaccard = final_df.iloc[i]['jaccard']
      curr_jaccard_candidates = []
    
  return candidate_pairs


# Recommendation
def recommendation_from_features(df, input):
  exact_tags = set()
  for category in input['category']:
    exact_tags.add(category)
  
  jaccard_tags = set()
  for value in input.values():
    if type(value) == type('str'):
      jaccard_tags.add(value)
    else:
      for subvalue in value:
        jaccard_tags.add(subvalue)
  
  candidate_restaurant, candidate_pairs = exact_match(df, exact_tags)
  # If candidate size from exact match is more than 10, choose a random sample
  if len(candidate_pairs) >= 10:
    final_candidates = random.sample(candidate_pairs, 10)
  # Else use jaccard match to fill the spots
  else:
    df = remaining_df(df, candidate_restaurant)
    final_candidates = jaccard_match(df, jaccard_tags, candidate_pairs)

  return final_candidates

In [23]:
input1 = {'risk_level': 'LOW',
         'price': '$$',
         'category': ['Shaved Ice', 'Desserts'],
         'delivery': 'offers delivery',
         'takeout': 'offers takeout',
          'group': 'good for groups',
         'vegetarian': 'offers vegetarian options',
         'reservation': 'offers reservation',
          'wheelchair': 'wheelchair accessible'}
recommendation_from_features(data_key, input1)

<ipython-input-22-13d1b6201005>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['jaccard'] = df.apply(lambda row: jaccard_similarity(row, tags), axis = 1)


[('Anko', 'https://www.yelp.com/biz/anko-los-angeles', 1.0),
 ('Oakobing',
  'https://www.yelp.com/biz/oakobing-los-angeles?osq=Restaurants',
  1.0),
 ('Natalee Thai Restaurant',
  'https://www.yelp.com/biz/natalee-thai-restaurant-los-angeles-4?osq=Restaurants',
  0.6154),
 ('Genwa Korean BBQ Mid Wilshire',
  'https://www.yelp.com/biz/genwa-korean-bbq-mid-wilshire-los-angeles?osq=Restaurants',
  0.5333),
 ('Chin Chin',
  'https://www.yelp.com/biz/chin-chin-brentwood?osq=Restaurants',
  0.5333),
 ('Toranj',
  'https://www.yelp.com/biz/toranj-los-angeles-2?osq=Restaurants',
  0.5),
 ('Tawanna Thai',
  'https://www.yelp.com/biz/tawanna-thai-los-angeles?osq=Restaurants',
  0.5),
 ('Noree Thai on Beverly',
  'https://www.yelp.com/biz/noree-thai-on-beverly-los-angeles?osq=Restaurants',
  0.5),
 ('Taj India Palace',
  'https://www.yelp.com/biz/taj-india-palace-los-angeles?osq=Restaurants',
  0.5),
 ('Hollywood Thai',
  'https://www.yelp.com/biz/hollywood-thai-los-angeles-2?osq=Restaurants',
 

In [ ]:
# get unique categories
def get_category(data):
    categories = set()
    for x in list(data.category):
        # 'Korean, Barbeque'
        clists = x.split(', ')
        print(clists)
        for c in clists:
            categories.add(c)
    return list(categories)
  

get_category(data_key)

In [ ]:
# Demo #2 - recommendation based on current resuaurant

# Choose candidates by calculating jaccard similarities
def jaccard_top10(df, tags):

  # Calculate jaccard similarity between input tags and each restaurant tags
  df['jaccard'] = df.apply(lambda row: jaccard_similarity(row, tags), axis = 1)
  final_df = df.sort_values(by = ['jaccard'], ascending = False).reset_index(drop = True)

  # Choose the top 10-len(candidates) jaccard similarities
  candidate_pairs = []
  i = 0
  curr_jaccard_candidates = []
  curr_jaccard = final_df.iloc[0]['jaccard']

  while i < len(final_df) and len(candidate_pairs) < 11:
    # If there are ties, record a list of restaurants with the current jaccard similarity
    if final_df.iloc[i]['jaccard'] == curr_jaccard:
      curr_jaccard_candidates.append((final_df.iloc[i]['name'], final_df.iloc[i]['url'], final_df.iloc[i]['jaccard']))
      i += 1
    # As soon as a different, lower jaccard similarity appears
    else:
      # If the recorded list can fit into the remaining spots of candidate_pairs, add all pairs in
      if len(curr_jaccard_candidates) <= 11 - len(candidate_pairs):
        for pair in curr_jaccard_candidates:
          if pair not in candidate_pairs:
            candidate_pairs.append(pair)
      # Else choose a random sample that can fit into the remaining spots
      else:
        new_candidates = random.sample(curr_jaccard_candidates, 11 - len(candidate_pairs))
        for pair in new_candidates:
          if pair not in candidate_pairs:
            candidate_pairs.append(pair)

      # Reset that different, lower jaccard similariy as the current value
      curr_jaccard = final_df.iloc[i]['jaccard']
      curr_jaccard_candidates = []
    
  return candidate_pairs

# Recommendation
def recommendation_from_restaurant(df, input):
  tags = set(df[df['name'] == input]['tags'].tolist()[0])
  final_candidates = jaccard_top10(df, tags)
  return final_candidates

In [ ]:
input2 = 'Moon BBQ 2'
recommendation_from_restaurant(data_key, input2)

<ipython-input-30-a41cb35092e7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['jaccard'] = df.apply(lambda row: jaccard_similarity(row, tags), axis = 1)


[('Moon BBQ 2',
  'https://www.yelp.com/biz/moon-bbq-2-los-angeles-2?osq=Restaurants',
  1.0),
 ('Jaragua',
  'https://www.yelp.com/biz/jaragua-los-angeles?osq=Restaurants',
  0.7692),
 ('Ham Ji Park', 'https://www.yelp.com/biz/ham-ji-park-los-angeles-2', 0.7143),
 ('Ahgassi Gopchang',
  'https://www.yelp.com/biz/ahgassi-gopchang-los-angeles-6',
  0.7143),
 ('Jae Bu Do',
  'https://www.yelp.com/biz/jae-bu-do-los-angeles-4?osq=Restaurants',
  0.7143),
 ('Wi Korean BBQ',
  'https://www.yelp.com/biz/wi-korean-bbq-los-angeles?osq=Restaurants',
  0.7143),
 ('Sun Ha Jang',
  'https://www.yelp.com/biz/sun-ha-jang-los-angeles?osq=Restaurants',
  0.7143),
 ('Kang Ho-dong Baekjeong',
  'https://www.yelp.com/biz/kang-ho-dong-baekjeong-los-angeles-4',
  0.7143),
 ('Yerim Korean BBQ',
  'https://www.yelp.com/biz/yerim-korean-bbq-los-angeles-2?osq=Restaurants',
  0.7143),
 ('Galbi King',
  'https://www.yelp.com/biz/galbi-king-los-angeles?osq=Restaurants',
  0.6667),
 ('Quarters Korean BBQ',
  'https

In [ ]:
res = recommendation_from_restaurant(data_key, input2)

<ipython-input-30-a41cb35092e7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['jaccard'] = df.apply(lambda row: jaccard_similarity(row, tags), axis = 1)


In [ ]:
data_key.head()

,name,SCORE,size,risk_level,url,score,review_counts,price,category,delivery,takeout,groups,vegetarian,reservation,wheelchair,address,zipcode,tags,jaccard
0,One Cafe,90.521739,0-30,HIGH,https://www.yelp.com/biz/one-cafe-los-angeles-...,3.5,7 reviews,$$,Cafes,1.0,1.0,0.0,NaN,0.0,0.0,"2080 Century Park E Los Angeles, CA 90067",90067,"[0-30, HIGH, $$, Cafes, offers delivery, offer...",0.2105
1,Moon BBQ 2,91.880952,61-150,HIGH,https://www.yelp.com/biz/moon-bbq-2-los-angele...,3.0,838 reviews,$$,"Korean, Barbeque",0.0,0.0,1.0,1.0,0.0,0.0,"478 N Western Ave Los Angeles, CA 90004",90004,"[61-150, HIGH, $$, Korean, Barbeque, no delive...",1.0000
2,101 Asian Kitchen,90.894737,0-30,HIGH,https://www.yelp.com/biz/101-asian-kitchen-los...,3.5,232 reviews,$$,"Chinese, Sushi Bars, Hawaiian",1.0,1.0,1.0,NaN,0.0,1.0,"7170 Beverly Blvd Los Angeles, CA 90036",90036,"[0-30, HIGH, $$, Chinese, Sushi Bars, Hawaiian...",0.2500
3,101 Asian Kitchen,90.000000,0-30,HIGH,https://www.yelp.com/biz/101-asian-kitchen-los...,3.5,232 reviews,$$,"Chinese, Sushi Bars, Hawaiian",1.0,1.0,1.0,NaN,0.0,1.0,"7170 Beverly Blvd Los Angeles, CA 90036",90036,"[0-30, HIGH, $$, Chinese, Sushi Bars, Hawaiian...",0.2500
4,5959 Kitchen and Bar,99.000000,"2,000+ SF",LOW,https://www.yelp.com/biz/5959-kitchen-and-bar-...,5.0,2 reviews,NaN,"Cocktail Bars, American (Traditional)",0.0,1.0,NaN,NaN,NaN,1.0,"5959 W Century Blvd Los Angeles, CA 90045",90045,"[2,000+ SF, LOW, nan, Cocktail Bars, American ...",0.1429


In [ ]:
# Evaluation...